In [1]:
import torch
from PIL import ImageFont, ImageDraw, Image
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import easyocr
import cv2
import matplotlib.pyplot as plt
import math
from sklearn.metrics import r2_score

CKPT_PATH = './best.pt'
yolov5 = torch.hub.load('ultralytics/yolov5',
                        'custom',
                        path=CKPT_PATH)

'confidence: ' + str(yolov5.conf)
reader = easyocr.Reader(['ko'])

/Users/idahyeon/q/envs/newenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /Users/idahyeon/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-8 Python-3.8.13 torch-1.12.1 CPU

Fusing layers... 
Model summary: 378 layers, 35248920 parameters, 0 gradients
Adding AutoShape... 
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


# 초반 상헌함수 사용 거리 가시화

In [5]:
cap = cv2.VideoCapture("./blur/blur_2_2.mp4")
outputFile = "blur2_2_result_test.mp4"
font=ImageFont.truetype(os.path.join("./font/Nanum_Gothic/NanumGothic-Bold.ttf"),20)
vid_writer = cv2.VideoWriter(outputFile, cv2.VideoWriter_fourcc('D', 'I', 'V', 'X'), cap.get(cv2.CAP_PROP_FPS), (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))



reader = easyocr.Reader(['ko'])
while cv2.waitKey(1) <0:
    result =[]
    hasFrame, frame = cap.read()
    pred_df = pd.DataFrame()
    if not hasFrame :
        print("Done Processing !!")
        cv2.waitKey(3000)
        break
    pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
    for i, row in pred.pandas().xyxy[0].iterrows():
        if row['confidence'] < yolov5.conf: break
        pred_df = pred_df.append(row)
    pred_df = pred_df.reset_index()
    pred.save()
    cv2.imwrite("frame.jpg",frame)
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    for i, row in pred_df.iterrows():
        img = np.asarray(frame)
        cnt=0
        imageText=img.copy()
        img_cropped = img[int(row['ymin']):int(row['ymax']), int(row['xmin']):int(row['xmax'])]
        img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
        plt.subplot(4,4,i+1)
        
        #거리구하기
        area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)
        distance = round(135/math.sqrt(area),2)
        
        draw.text((int(row['xmin'])+i*150,int(row['ymax'])+10),(str(distance)+" m"),font=font,fill=(0,0,255))
        
        
    img = np.array(img_pil)
    vid_writer.write(img)
    print(result)


vid_writer.release()
cap.release()


OpenCV: Couldn't read video stream from file "./blur/blur_2_2.mp4"
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Done Processing !!


# grd 함수 사용, 새로운 상수 사용 거리 가시화

In [5]:
##가로, 세로 길이 기반 측정
def grd_length(resolution, width, height):
    #normalize value
    mean = 520.2006621958521
    std = 675.1471686772571

    #normalizing
    resolution = (resolution-mean) / std
    height = (height-mean) / std
    width = (width-mean) / std
    
    #calculating distance
    distance = (-32.62872226) + (1.36456181*resolution) + (3.83042209*width) + ((-52.93560358)*height)
    
    return distance



##넓이 기반 측정
def grd_size(resolution, size):
    #normalize value
    mean = 708.7067193723738
    std = 750.7078400469876

    #normalizing
    resolution = (resolution-mean) / std
    size = (size-mean) / std
    
    #calculating distance
    distance = (-12.97574844) + (1.36510736*resolution) + ((-19.48221397)*size)
    
    return distance

def grd_length_resolution(resolution, width, height):
    distance = 0
    
    if resolution == 720:
        #normalize value
        mean = 67.82630844999002
        std = 61.346605511051685
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-18.58963292) + (5.40696279*width) + ((-29.74396128)*height)
    
    elif resolution == 1080:
        #normalize value
        mean = 100.09815308569979
        std = 90.0449028230475
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-5.48210169) + (1.38873641*width) + ((-13.10648836)*height)
        
    elif resolution == 2160:
        #normalize value
        mean = 192.97851834564483
        std = 173.24456702735597
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-2.01397684) + (0.425009*width) + ((-8.78440094)*height)
        
    
    
    return distance
    

def grd_size_resolution(resolution, size):
    distance = 0
    
    if resolution == 720:
        #normalize value
        mean = 55.162371024747586
        std = 31.419484714306954
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.68509687)*size)
    
    elif resolution == 1080:
        #normalize value
        mean = 80.87600296910803
        std = 44.903575372439505
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.67382192)*size)
        
    elif resolution == 2160:
        #normalize value
        mean = 156.2019422403862
        std = 87.05443271571005
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.68307815)*size)
        
    
    
    return distance
    
cap = cv2.VideoCapture("./720p/700cm_720p.MOV")
outputFile = "grd_result_7m.mp4"
font=ImageFont.truetype(os.path.join("./font/Nanum_Gothic/NanumGothic-Bold.ttf"),20)
vid_writer = cv2.VideoWriter(outputFile, cv2.VideoWriter_fourcc('D', 'I', 'V', 'X'), cap.get(cv2.CAP_PROP_FPS), (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

width = cv2.CAP_PROP_FRAME_WIDTH
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

reader = easyocr.Reader(['ko'])
while cv2.waitKey(1) <0:
    result =[]
    hasFrame, frame = cap.read()
    pred_df = pd.DataFrame()
    if not hasFrame :
        print("Done Processing !!")
        cv2.waitKey(3000)
        break
    pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
    for i, row in pred.pandas().xyxy[0].iterrows():
        if row['confidence'] < yolov5.conf: break
        pred_df = pred_df.append(row)
    pred_df = pred_df.reset_index()
    pred.save()
    cv2.imwrite("frame.jpg",frame)
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    for i, row in pred_df.iterrows():
        img = np.asarray(frame)
        cnt=0
        imageText=img.copy()
        img_cropped = img[int(row['ymin']):int(row['ymax']), int(row['xmin']):int(row['xmax'])]
        img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
        plt.subplot(4,4,i+1)
        
        #거리구하기
        # area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)
        area = (row['ymax']-row['ymin'])*(row['xmax']-row['xmin'])
        # distance = round(135/math.sqrt(area),2)
        
        distance1 = grd_length(float(height),(row['xmax']-row['xmin']),(row['ymax']-row['ymin']))
        # distance2 = grd_size(float(height),math.sqrt(abs(row['xmax']-row['xmin'])*abs(row['ymax']-row['ymin'])))
        distance2 = grd_size(float(height),math.sqrt(area))
        distance3 = grd_length_resolution(float(height),(row['xmax']-row['xmin']),row['ymax']-row['ymin'])
        distance4 = grd_size_resolution(float(height),math.sqrt(area))
        
        if(float(720)==height):
            distance = round(167/math.sqrt(area),2)
        elif(float(1080)==height):
            distance = round(248/math.sqrt(area),2)
        elif(float(2160)==height):
            distance = round(478/math.sqrt(area),2)
        
        print("--------------")
        print("height: "+str(height))
        print("가로, 세로 : "+str(row['xmax']-row['xmin'])+", "+str(row['ymax']-row['ymin']))
        print("넓이: "+str(math.sqrt((row['xmax']-row['xmin'])*(row['ymax']-row['ymin']))))
        print("grd_length: "+str(distance1))
        print("grd_size: "+str(distance2))
        print("grd_length_resolution: "+str(distance3))
        print("grd_size_resolution: "+str(distance4))
        print("상헌 상수 거리: "+str(distance))
        print("실제 거리 : 7m")
        print("--------------")
        
        # draw.text(int(row['xmin'])+i*150,int(row['ymax'])+10),(str(distance1)+" m"),font=font,fill=(0,0,255))
        # draw.text((int(row['xmin'])+i*150,int(row['ymax'])+10),(str(distance)+" m"),font=font,fill=(0,0,255)) 
        
    img = np.array(img_pil)
    vid_writer.write(img)
    print(result)


vid_writer.release()
cap.release()


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_40974/472823878.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp3277


--------------
height: 720.0
가로, 세로 : 44.96044921875, 14.231201171875
넓이: 25.29508247881199
grd_length: 4.749893720839914
grd_size: 4.780544446480723
grd_length_resolution: 5.380657422928504
grd_size_resolution: 5.601849136030717
상헌 상수 거리: 6.6
실제 거리 : 7m
--------------
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_40974/472823878.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp3278


--------------
height: 720.0
가로, 세로 : 45.459228515625, 14.13629150390625
넓이: 25.35004745241244
grd_length: 4.760165015926404
grd_size: 4.779118006900083
grd_length_resolution: 5.470635842138531
grd_size_resolution: 5.598901242205725
상헌 상수 거리: 6.59
실제 거리 : 7m
--------------
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_40974/472823878.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp3279


...

/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_40974/472823878.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp3284


--------------
height: 720.0
가로, 세로 : 43.73583984375, 13.779052734375
넓이: 24.548695354116287
grd_length: 4.778397105787668
grd_size: 4.799914530143182
grd_length_resolution: 5.491947497383766
grd_size_resolution: 5.641879538154413
상헌 상수 거리: 6.8
실제 거리 : 7m
--------------
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_40974/472823878.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp3285


--------------
height: 720.0
가로, 세로 : 45.3763427734375, 13.93646240234375
넓이: 25.147280071965522
grd_length: 4.775362571662789
grd_size: 4.784380183849981
grd_length_resolution: 5.560217798464915
grd_size_resolution: 5.609776107953399
상헌 상수 거리: 6.64
실제 거리 : 7m
--------------
[]


KeyboardInterrupt: 

# m 별 xmax-xmin , ymax-ymin 구하기

In [ ]:
f=open("./4k/4k_m.txt",'w')


vedios = os.listdir('./4k/')
vedios.sort()

for pth in vedios:
    print("==================")
    print("pth : " + str(pth))
    y_distance=[]
    x_distance=[]
    
    cap = cv2.VideoCapture("./4k/"+str(pth))
    
    reader = easyocr.Reader(['ko']) #이미지 -> 텍스트화 
    while cv2.waitKey(1) <0:
        
        hasFrame, frame = cap.read()
        pred_df = pd.DataFrame()
        if not hasFrame :
            print("Done Processing !!")
            cv2.waitKey(3000)
            break
        pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
        for i, row in pred.pandas().xyxy[0].iterrows():
            if row['confidence'] < yolov5.conf: break
            pred_df = pred_df.append(row)
        pred_df = pred_df.reset_index()
        pred.save()
        cv2.imwrite("frame.jpg",frame)
        img_pil = Image.fromarray(frame)
        draw = ImageDraw.Draw(img_pil)
        for i, row in pred_df.iterrows():
            img = np.asarray(frame)
            cnt=0
            imageText=img.copy()
            img_cropped = img[int(row['ymin']):int(row['ymax']), int(row['xmin']):int(row['xmax'])]
            img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
            plt.subplot(4,4,i+1)
            
            y_distance.append(row['ymax']-row['ymin'])
            x_distance.append(row['xmax']-row['xmin'])
            
            # print("ymax-ymin : "+str(row['ymax']-row['ymin']))
            # print("xmax-xmin : "+str(row['xmax']-row['xmin']))
            
    cap.release()
    result_mean = "2160 "+str(np.mean(x_distance))+ " " +str(np.mean(y_distance)) +"\n"
    print(result_mean)
    f.write(result_mean)
f.close()


# 거리구하기 가시화

In [3]:
cap = cv2.VideoCapture("./total_vedio/26.mp4")
outputFile = "result_heemin.mp4"
font=ImageFont.truetype(os.path.join("./font/Nanum_Gothic/NanumGothic-Bold.ttf"),50)
vid_writer = cv2.VideoWriter(outputFile, cv2.VideoWriter_fourcc('D', 'I', 'V', 'X'), cap.get(cv2.CAP_PROP_FPS), (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))



reader = easyocr.Reader(['ko'])
while cv2.waitKey(1) <0:
    result =[]
    hasFrame, frame = cap.read()
    pred_df = pd.DataFrame()
    if not hasFrame :
        print("Done Processing !!")
        cv2.waitKey(3000)
        break
    pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
    for i, row in pred.pandas().xyxy[0].iterrows():
        if row['confidence'] < yolov5.conf: break
        pred_df = pred_df.append(row)
    pred_df = pred_df.reset_index()
    pred.save()
    cv2.imwrite("frame.jpg",frame)
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    for i, row in pred_df.iterrows():
        img = np.asarray(frame)
        cnt=0
        imageText=img.copy()
        img_cropped = img[int(row['ymin']):int(row['ymax']), int(row['xmin']):int(row['xmax'])]
        img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
        plt.subplot(4,4,i+1)
        
        #거리구하기
        area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)
        distance = round(135/math.sqrt(area),2)
        
        print(distance)
        
        draw.text((int(row['xmin'])+i*150,int(row['ymax'])+10),(str(distance)+" m"),font=font,fill=(0,0,255))
        
        break
        
        
    img = np.array(img_pil)
    vid_writer.write(img)
    print(result)


vid_writer.release()
cap.release()


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_33315/1841487653.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp5974


0.94
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_33315/1841487653.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp5975


0.95
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_33315/1841487653.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp5976


...

0.96
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_33315/1841487653.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp6167


0.96
[]


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_33315/1841487653.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp6168


0.95
[]
Done Processing !!


# 거리 정확도 구하기 ( R-Squared )

In [14]:
def getSST(m,results):
    result=0
    mean = np.mean(results)
    for r in results:
        result+= math.pow((m-mean),2)
    print("SST - "+str(m)+" : mean : "+ str(mean) + " ,result : "+str(result))
    return result

def getSSE(results):
    mean = np.mean(results)
    result=0
    for r in results:
        result+= math.pow((r-mean),2)
    print("SSE - mean : "+ str(mean) + " ,result : "+str(result))
    return result

def getSSR(m,results):
    result=0
    for r in results:
        result += math.pow((m-r),2)
    print("SSR- "+str(m)+" : result : "+str(result))
    return result


one_result=[]  #1m
three_result=[] #3m
five_result=[] #5m
seven_result=[] #7m

one = [str(i) for i in range(1,9)]
three = [str(i) for i in range(9,17)]
five = [str(i) for i in range(17,25)]
seven = [str(i) for i in range(25,33)]

vedios = os.listdir('./data/')
for pth in vedios:
    print("==================")
    print("pth : " + str(pth))
    
    cap = cv2.VideoCapture("./data/"+str(pth))
    
    reader = easyocr.Reader(['ko']) #이미지 -> 텍스트화 
    
    while cv2.waitKey(1) <0:
        hasFrame, frame = cap.read()
        pred_df = pd.DataFrame()
        
        if not hasFrame :   #프레임 끝났을 경우 종료 
            # print("Done Processing !!")
            cv2.waitKey(3000)
            break
        
        pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
        
        for i, row in pred.pandas().xyxy[0].iterrows(): #객체 정확도 분별
            if row['confidence'] < yolov5.conf: break
            pred_df = pred_df.append(row)
            
        pred_df = pred_df.reset_index()
        pred.save()
        
        for i, row in pred_df.iterrows(): # 한 프레임에서 인식한 객체 
            #거리구하기
            area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)  #넓이
            distance = round(135/math.sqrt(area),2)  #135 : 상헌상수
            
            if any(x in pth for x in seven) : 
                seven_result.append(distance)
            elif any(x in pth for x in five) : 
                five_result.append(distance)
            elif any(x in pth for x in three) : 
                three_result.append(distance)
            elif any(x in pth for x in one) : 
                one_result.append(distance)
            else : print("error pth")
            
            break
            
    cap.release()

SST = getSST(1,one_result)+getSST(3,three_result)+getSST(5,five_result)+getSST(7,seven_result)
SSE = getSSE(one_result)+getSSE(three_result)+getSSE(five_result)+getSSE(seven_result)
SSR = getSSR(1,one_result)+getSSR(3,three_result)+getSSR(5,five_result)+getSSR(7,seven_result)

print("SST: "+ str(SST))
print("SSE: "+ str(SSE))
print("SSR: "+ str(SSR))

one = [1 for i in range(len(one_result))]
three = [3 for i in range(len(three_result))]
five = [5 for i in range(len(five_result))]
seven = [7 for i in range(len(seven_result))]

total = one+three+five+seven

total_result = one_result+three_result+five_result+seven_result

print("rs 라이브러리 : "+ str(r2_score(total, total_result)))


R_Squared1= SSE/SST
R_Squared2 = 1 - (SSR/SST)

print("SSE/SST : "+str(R_Squared1))
print("1- (SSR/SST) : "+str(R_Squared2))
print("1-(SSE/SST) : "+str(1-(SSE/SST)))
print("---SST 값 변경----")
print("SSE/SST : "+str(SSE/SSE+SSR))
print("1- (SSR/SST) : "+str(1-(SSR/SSE+SSR)))
print("1-(SSE/SST) : "+str(1-(SSE/SSE+SSR)))

FileNotFoundError: [Errno 2] No such file or directory: './data/'

In [45]:
print("rs 라이브러리 :  "+str(r2_score(total, total_result)))

def getSST1(correct_results,y_mean):
    result=0
    for c in correct_results:
        result+= math.pow((c-y_mean),2)
    print("SST - : mean : "+ str(y_mean) + ",result : "+str(result))
    return result

def getSSE1(y_mean,results):
    result=0
    for r in results:
        result+= math.pow((r-(y_mean)),2)
    print("SSE - mean : "+ str(y_mean) + ",result : "+str(result))
    return result

def getSSR1(correct_results,results):
    result=0
    for i in range(len(correct_results)):
        result += math.pow((correct_results[i]-results[i]),2)
    print("SSR- : result : "+str(result))
    return result

# y_mean = (1*len(one_result)+3*len(three_result)+5*len(five_result)+7*len(seven_result))/(len(one_result)+len(three_result)+len(five_result)+len(seven_result))
y_mean = np.mean(total)
SST = getSST1(total,y_mean)
SSE = getSSE1(y_mean,total_result)
SSR = getSSR1(total,total_result)

print("SST: "+ str(SST))
print("SSE: "+ str(SSE))
print("SSR: "+ str(SSR))


print("SSE/SST : "+str(SSE/SST))
print("r: "+str(math.sqrt(SSE/SST)))
print("1- (SSR/SST) : "+str(1 - (SSR/SST)))
print("---SST 값 변경----")
print("SSE/SST : "+str(SSE/(SSE+SSR)))
print("1- (SSR/SST) : "+str(1-(SSR/(SSE+SSR))))


# print("SSR: "+str(np.sum((total_result - np.mean(total))**2)))
# print("SSE: "+str(np.sum((total - total_result)**2)))
# SSTO = SSR+SSE

# print(SSR/SSTO, 1-SSE/SSTO)


rs 라이브러리 :  0.7014997402137064
SST - : mean : 3.957692307692308,result : 10156.276923077152
SSE - mean : 3.957692307692308,result : 16407.677069230802
SSR- : result : 3031.651299999996
SST: 10156.276923077152
SSE: 16407.677069230802
SSR: 3031.651299999996
SSE/SST : 1.6155208442524034
r: 1.2710314096246416
1- (SSR/SST) : 0.7014997402137135
---SST 값 변경----
SSE/SST : 0.8440454709948421
1- (SSR/SST) : 0.844045470994842


In [43]:
from sklearn.metrics import mean_absolute_error
print("MAE: "+str(mean_absolute_error(total, total_result)))
print("MAE 1m : "+str(mean_absolute_error(one, one_result)))
print("MAE 3m : "+str(mean_absolute_error(three, three_result)))
print("MAE 5m : "+str(mean_absolute_error(five, five_result)))
print("MAE 7m : "+str(mean_absolute_error(seven, seven_result)))

from sklearn.metrics import mean_squared_error 
print("MSE: "+str(mean_squared_error(total, total_result)))

from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(total, total_result) 
print("RMSE: "+str(np.sqrt(MSE)))

print("rs 라이브러리 :  "+str(r2_score(total, total_result)))

print(len(total_result))



MAE: 0.8733317307692309
MAE 1m : 0.19596491228070173
MAE 3m : 0.5325225225225225
MAE 5m : 1.15926213592233
MAE 7m : 1.656800804828974
MSE: 1.4575246634615382
RMSE: 1.2072798612838442
rs 라이브러리 :  0.7014997402137064
2080


# 거리 정확도 구하기 (새로운 상수)

In [6]:
total_result=[]
total=[]
result_720=[]
result_1080=[]
result_2160=[]

vedios = os.listdir('./pixel_vedio/')
vedios.sort()

for pth in vedios:
    print("==================")
    print("pth : " + str(pth))
    
    
    cap = cv2.VideoCapture("./pixel_vedio/"+str(pth))
    
    
    width = cv2.CAP_PROP_FRAME_WIDTH
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    
    if "100" in pth : m=1
    elif "150" in pth : m=1.5
    elif "200" in pth : m=2
    elif "250" in pth : m=2.5
    elif "300" in pth : m=3
    elif "350" in pth : m=3.5
    elif "400" in pth : m=4
    elif "450" in pth : m=4.5
    elif "500" in pth : m=5
    elif "550" in pth : m=5.5
    elif "600" in pth : m=6
    elif "650" in pth : m=6.5
    elif "700" in pth : m=7
    
    if pth==".DS_Store": continue
    while cv2.waitKey(1) <0:
        hasFrame, frame = cap.read()
        pred_df = pd.DataFrame()
        
        if not hasFrame :   #프레임 끝났을 경우 종료 
            # print("Done Processing !!")
            cv2.waitKey(3000)
            break
        
        pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
        
        for i, row in pred.pandas().xyxy[0].iterrows(): #객체 정확도 분별
            if row['confidence'] < yolov5.conf: break
            pred_df = pred_df.append(row)
            
        pred_df = pred_df.reset_index()
        pred.save()
        
        for i, row in pred_df.iterrows(): # 한 프레임에서 인식한 객체 
            #거리구하기
            area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)  #넓이
            
            if(float(720)==height):
                distance = round(167/math.sqrt(area),2)
                print("상수 : 167")
            elif(float(1080)==height):
                distance = round(248/math.sqrt(area),2)
                print("상수 : 248")
            elif(float(2160)==height):
                distance = round(478/math.sqrt(area),2)
                print("상수 : 478")
            else: distance=0
            
            total_result.append(distance)
            total.append(m)
            
            print("예측 거리 : "+ str(distance))
            print("실제 거리 : "+ str(m))
            
    cap.release()

# total = [1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]

# total_result = one_result+three_result+five_result+seven_result

print("rs 라이브러리 : "+ str(r2_score(total, total_result)))



pth : .DS_Store
pth : 100cm_720p.MOV


OpenCV: Couldn't read video stream from file "./720p/.DS_Store"
[ERROR:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap.cpp (166) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): ./720p/.DS_Store in function 'icvExtractPattern'


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_12857/3007625093.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp835


total_result : 1.29
m : 1


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_12857/3007625093.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp836


total_result : 1.3
m : 1


...

total_result : 11.45
m : 7


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_12857/3007625093.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp1666


total_result : 11.38
m : 7


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_12857/3007625093.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp1667


total_result : 11.39
m : 7
rs 라이브러리 : -0.1866260465677405


In [7]:
print(len(total),len(total_result))

833 833


In [9]:
from sklearn.metrics import mean_absolute_error
print("MAE: "+str(mean_absolute_error(total, total_result)))

from sklearn.metrics import mean_squared_error 
print("MSE: "+str(mean_squared_error(total, total_result)))

from sklearn.metrics import mean_squared_error 
MSE = mean_squared_error(total, total_result) 
print("RMSE: "+str(np.sqrt(MSE)))

print("rs 라이브러리 :  "+str(r2_score(total, total_result)))

print(len(total_result))

MAE: 1.5639615846338533
MSE: 4.167478271308523
RMSE: 2.041440244363896
rs 라이브러리 :  -0.1866260465677405
833


# 거리 정확도 _ 새로운 상수 (전체 화질 다 합쳐서 결과 내기)

In [3]:
##가로, 세로 길이 기반 측정
def grd_length(resolution, width, height):
    #normalize value
    mean = 520.2006621958521
    std = 675.1471686772571

    #normalizing
    resolution = (resolution-mean) / std
    height = (height-mean) / std
    width = (width-mean) / std
    
    #calculating distance
    distance = (-32.62872226) + (1.36456181*resolution) + (3.83042209*width) + ((-52.93560358)*height)
    
    return distance



##넓이 기반 측정
def grd_size(resolution, size):
    #normalize value
    mean = 708.7067193723738
    std = 750.7078400469876

    #normalizing
    resolution = (resolution-mean) / std
    size = (size-mean) / std
    
    #calculating distance
    distance = (-12.97574844) + (1.36510736*resolution) + ((-19.48221397)*size)
    
    return distance

def grd_length_resolution(resolution, width, height):
    distance = 0
    
    if resolution == 720:
        #normalize value
        mean = 67.82630844999002
        std = 61.346605511051685
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-18.58963292) + (5.40696279*width) + ((-29.74396128)*height)
    
    elif resolution == 1080:
        #normalize value
        mean = 100.09815308569979
        std = 90.0449028230475
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-5.48210169) + (1.38873641*width) + ((-13.10648836)*height)
        
    elif resolution == 2160:
        #normalize value
        mean = 192.97851834564483
        std = 173.24456702735597
        
        #normalizing
        width = (width-mean) / std
        height = (height-mean) / std
        
        #calculating distance
        distance = (-2.01397684) + (0.425009*width) + ((-8.78440094)*height)
        
    
    
    return distance
    
def grd_size_resolution(resolution, size):
    distance = 0
    
    if resolution == 720:
        #normalize value
        mean = 55.162371024747586
        std = 31.419484714306954
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.68509687)*size)
    
    elif resolution == 1080:
        #normalize value
        mean = 80.87600296910803
        std = 44.903575372439505
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.67382192)*size)
        
    elif resolution == 2160:
        #normalize value
        mean = 156.2019422403862
        std = 87.05443271571005
        
        #normalizing
        size = (size-mean) / std
        
        #calculating distance
        distance = (4) + ((-1.68307815)*size)
        
    
    
    return distance
    

# total_result=[]
total=[]

# total_result_720=[]  #새로운 상수 - 예측값
# total_result_1080=[] #새로운 상수 - 예측값
# total_result_2160=[] #새로운 상수 - 예측값
# total_720=[] #새로운 상수 - 관측값
# total_1080=[] #새로운 상수 - 관측값
# total_2160=[] #새로운 상수 - 관측값

total_result_grd_length=[]
total_result_grd_size=[]
total_result_grd_length_resolution=[]
total_result_grd_size_resolution=[]
total_result_new_con=[]


vedios = os.listdir('./pixel_vedio/')
vedios.sort()

for pth in vedios:
    print("==================")
    print("pth : " + str(pth))
    
    
    cap = cv2.VideoCapture("./pixel_vedio/"+str(pth))
    
    
    width = cv2.CAP_PROP_FRAME_WIDTH
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    
    if "100" in pth : m=1
    elif "150" in pth : m=1.5
    elif "200" in pth : m=2
    elif "250" in pth : m=2.5
    elif "300" in pth : m=3
    elif "350" in pth : m=3.5
    elif "400" in pth : m=4
    elif "450" in pth : m=4.5
    elif "500" in pth : m=5
    elif "550" in pth : m=5.5
    elif "600" in pth : m=6
    elif "650" in pth : m=6.5
    elif "700" in pth : m=7
    
    if pth==".DS_Store": continue
    while cv2.waitKey(1) <0:
        hasFrame, frame = cap.read()
        pred_df = pd.DataFrame()
        
        if not hasFrame :   #프레임 끝났을 경우 종료 
            # print("Done Processing !!")
            cv2.waitKey(3000)
            break
        
        pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
        
        for i, row in pred.pandas().xyxy[0].iterrows(): #객체 정확도 분별
            if row['confidence'] < yolov5.conf: break
            pred_df = pred_df.append(row)
            
        pred_df = pred_df.reset_index()
        pred.save()
        
        for i, row in pred_df.iterrows(): # 한 프레임에서 인식한 객체 
            #거리구하기
            # area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)  #넓이
            area = (row['ymax']-row['ymin'])*(row['xmax']-row['xmin'])  #넓이
            
            
            total_result_grd_length.append(grd_length(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin']))
            total_result_grd_size.append(grd_size(float(height),math.sqrt(area)))
            total_result_grd_length_resolution.append(grd_length_resolution(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin']))
            total_result_grd_size_resolution.append(grd_size_resolution(float(height),math.sqrt(area)))
            
            if(float(720)==height):
                distance = round(167/math.sqrt(area),2)
                print("상수 : 167")
            elif(float(1080)==height):
                distance = round(248/math.sqrt(area),2)
                print("상수 : 248")
            elif(float(2160)==height):
                distance = round(478/math.sqrt(area),2)
                print("상수 : 478")
            else: distance=0
            
            total_result_new_con.append(distance)
            
            total.append(m)
            
            print("실제 거리 : "+ str(m))
            print("grd_length : "+str(grd_length(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin'])))
            print("grd_size : "+str(grd_size(float(height),math.sqrt(area))))
            print("grd_length_resolution : "+str(grd_length_resolution(float(height),row['xmax']-row['xmin'],row['ymax']-row['ymin'])))
            print("grd_size_resolution : "+str(grd_size_resolution(float(height),math.sqrt(area))))
            print("상헌 상수 예측 거리 : "+ str(distance))
            
            
    cap.release()

print("rs 라이브러리 _ grd_length : "+ str(r2_score(total, total_result_grd_length)))
print("rs 라이브러리 _ grd_size : "+ str(r2_score(total, total_result_grd_size)))
print("rs 라이브러리 _ grd_length_resolution : "+ str(r2_score(total, total_result_grd_length_resolution)))
print("rs 라이브러리 _ grd_size_resolution : "+ str(r2_score(total, total_result_grd_size_resolution)))
print("rs 라이브러리 _ 새로운 상헌 상수 : "+ str(r2_score(total, total_result_new_con)))


from sklearn.metrics import mean_absolute_error
MAE_grd_length=mean_absolute_error(total, total_result_grd_length)
MAE_grd_size=mean_absolute_error(total, total_result_grd_size)
MAE_grd_length_resolution=mean_absolute_error(total, total_result_grd_length_resolution)
MAE_grd_size_resolution=mean_absolute_error(total, total_result_grd_size_resolution)
MAE_new_con=mean_absolute_error(total, total_result_new_con)
print("MAE - grd_length: "+str(mean_absolute_error(total, total_result_grd_length)))
print("MAE - grd_size : "+str(mean_absolute_error(total, total_result_grd_size)))
print("MAE - grd_length_resolution : "+str(mean_absolute_error(total, total_result_grd_length_resolution)))
print("MAE - grd_size_resolution : "+str(mean_absolute_error(total, total_result_grd_size_resolution)))
print("MAE - 새로운 상헌 상수 : "+str(mean_absolute_error(total, total_result_new_con)))

from sklearn.metrics import mean_squared_error 
MSE_grd_length=mean_squared_error(total, total_result_grd_length)
MSE_grd_size=mean_squared_error(total, total_result_grd_size)
MSE_grd_length_resolution=mean_squared_error(total, total_result_grd_length_resolution)
MSE_grd_size_resolution=mean_squared_error(total, total_result_grd_size_resolution)
MSE_new_con=mean_squared_error(total, total_result_new_con)
print("MSE - grd_length: "+str(mean_squared_error(total, total_result_grd_length)))
print("MSE - grd_size : "+str(mean_squared_error(total, total_result_grd_size)))
print("MSE - grd_length_resolution : "+str(mean_squared_error(total, total_result_grd_length_resolution)))
print("MSE - grd_size_resolution : "+str(mean_squared_error(total, total_result_grd_size_resolution)))
print("MSE - 새로운 상헌 상수 : "+str(mean_squared_error(total, total_result_new_con)))

from sklearn.metrics import mean_squared_error 
print("RMSE - grd_length: "+str(np.sqrt(MSE_grd_length)))
print("RMSE - grd_size: "+str(np.sqrt(MSE_grd_size)))
print("RMSE - grd_length_resolution: "+str(np.sqrt(MSE_grd_length_resolution)))
print("RMSE - grd_size_resolution: "+str(np.sqrt(MSE_grd_size_resolution)))
print("RMSE - 새로운 상헌 상수: "+str(np.sqrt(MSE_new_con)))


pth : .DS_Store
pth : 100_1080p.MOV


OpenCV: Couldn't read video stream from file "./pixel_vedio/.DS_Store"
[ERROR:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap.cpp (166) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): ./pixel_vedio/.DS_Store in function 'icvExtractPattern'


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_15070/1332704070.py:178: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp3286


상수 : 248
실제 거리 : 1
grd_length : 0.9696545843767055
grd_size : 0.9208331607399245
grd_length_resolution : -0.43805971766747576
grd_size_resolution : -0.4123600139920942
상헌 상수 예측 거리 : 1.24


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_15070/1332704070.py:178: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp3287


상수 : 248
실제 거리 : 1
grd_length : 1.045727285680762
grd_size : 0.9362584664058904
grd_length_resolution : -0.29075026246547414
grd_size_resolution : -0.3902038408678292
상헌 상수 예측 거리 : 1.25


...

상수 : 167
실제 거리 : 7
grd_length : 4.804113395124531
grd_size : 4.7883175348997025
grd_length_resolution : 5.766374865299021
grd_size_resolution : 5.617913075690299
상헌 상수 예측 거리 : 6.68


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_15070/1332704070.py:178: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp5777


상수 : 167
실제 거리 : 7
grd_length : 4.81522762365843
grd_size : 4.783009682489764
grd_length_resolution : 5.887885663080102
grd_size_resolution : 5.606943816595419
상헌 상수 예측 거리 : 6.63


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_15070/1332704070.py:178: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp5778


상수 : 167
실제 거리 : 7
grd_length : 4.803973127503227
grd_size : 4.786069250432336
grd_length_resolution : 5.778829282668212
grd_size_resolution : 5.613266749278634
상헌 상수 예측 거리 : 6.66
rs 라이브러리 _ grd_length : 0.6592636443191704
rs 라이브러리 _ grd_size : 0.6743953546457886
rs 라이브러리 _ grd_length_resolution : 0.8018907069310233
rs 라이브러리 _ grd_size_resolution : 0.8046925609549329
rs 라이브러리 _ 새로운 상헌 상수 : 0.9650820825687096
MAE - grd_length: 0.8840919011250645
MAE - grd_size : 0.8830531036742015
MAE - grd_length_resolution : 0.6791573846241201
MAE - grd_size_resolution : 0.7131221273304008
MAE - 새로운 상헌 상수 : 0.25035971223021575
MSE - grd_length: 1.1847839263026059
MSE - grd_size : 1.1321690324894036
MSE - grd_length_resolution : 0.6888513719362455
MSE - grd_size_resolution : 0.6791089668302754
MSE - 새로운 상헌 상수 : 0.12141406874500398
RMSE - grd_length: 1.088477802393143
RMSE - grd_size: 1.0640343192253734
RMSE - grd_length_resolution: 0.8299707054687204
RMSE - grd_size_resolution: 0.8240806798064589
RMSE 

# 거리 정확도 test

In [22]:
def getSST(m,results):
    result=0
    mean = np.mean(results)
    for r in results:
        result+= math.pow((m-mean),2)
    print("SST - "+str(m)+" : mean : "+ str(mean) + " ,result : "+str(result))
    return result

def getSSE(results):
    mean = np.mean(results)
    result=0
    for r in results:
        result+= math.pow((r-mean),2)
    print("SSE - mean : "+ str(mean) + " ,result : "+str(result))
    return result

def getSSR(m,results):
    result=0
    for r in results:
        result += math.pow((m-r),2)
    print("SSR- "+str(m)+" : result : "+str(result))
    return result


one_result=[]  #1m
three_result=[] #3m
five_result=[] #5m
seven_result=[] #7m

one = [str(i) for i in range(1,9)]
three = [str(i) for i in range(9,17)]
five = [str(i) for i in range(17,25)]
seven = [str(i) for i in range(25,33)]

vedios = os.listdir('./vedio_distance/')
for pth in vedios:
    print("==================")
    print("pth : " + str(pth))
    
    cap = cv2.VideoCapture("./vedio_distance/"+str(pth))
    
    reader = easyocr.Reader(['ko']) #이미지 -> 텍스트화 
    
    while cv2.waitKey(1) <0:
        hasFrame, frame = cap.read()
        pred_df = pd.DataFrame()
        
        if not hasFrame :   #프레임 끝났을 경우 종료 
            # print("Done Processing !!")
            cv2.waitKey(3000)
            break
        
        pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
        
        for i, row in pred.pandas().xyxy[0].iterrows(): #객체 정확도 분별
            if row['confidence'] < yolov5.conf: break
            pred_df = pred_df.append(row)
            
        pred_df = pred_df.reset_index()
        pred.save()
        
        for i, row in pred_df.iterrows(): # 한 프레임에서 인식한 객체 
            #거리구하기
            area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)  #넓이
            distance = round(135/math.sqrt(area),2)  #135 : 상헌상수
            
            if any(x in pth for x in seven) : 
                seven_result.append(distance)
            elif any(x in pth for x in five) : 
                five_result.append(distance)
            elif any(x in pth for x in three) : 
                three_result.append(distance)
            elif any(x in pth for x in one) : 
                one_result.append(distance)
            else : print("error pth")
            
            break
            
    cap.release()

SST = getSST(1,one_result)+getSST(3,three_result)+getSST(4,five_result)+getSST(7,seven_result)
SSE = getSSE(one_result)+getSSE(three_result)+getSSE(five_result)+getSSE(seven_result)
SSR = getSSR(1,one_result)+getSSR(3,three_result)+getSSR(4,five_result)+getSSR(7,seven_result)

print("SST: "+ str(SST))
print("SSE: "+ str(SSE))
print("SSR: "+ str(SSR))

one = [1 for i in range(len(one_result))]
three = [3 for i in range(len(three_result))]
five = [4 for i in range(len(five_result))]
seven = [7 for i in range(len(seven_result))]

total = one+three+five

total_result = one_result+three_result+five_result

# print("rs 라이브러리 1m : "+ str(r2_score(one, one_result)))
# print("rs 라이브러리 3m : "+ str(r2_score(three, three_result)))
# print("rs 라이브러리 4m : "+ str(r2_score(five, five_result)))
# print("rs 라이브러리 7m : "+ str(r2_score(seven, seven_result)))

print("rs 라이브러리 1m : "+ str(r2_score(total, total_result)))

print("++++++++++++")

R_Squared1= SSE/SST
R_Squared2 = 1 - (SSR/SST)

print("SSE/SST : "+str(R_Squared1))
print("1- (SSR/SST) : "+str(R_Squared2))
print("1-(SSE/SST) : "+str(1-(SSE/SST)))
print("---SST 값 변경----")
print("SSE/SST : "+str(SSE/SSE+SSR))
print("1- (SSR/SST) : "+str(1-(SSR/SSE+SSR)))
print("1-(SSE/SST) : "+str(1-(SSE/SSE+SSR)))

OpenCV: Couldn't read video stream from file "./vedio_distance/.DS_Store"
[ERROR:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap.cpp (166) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.4) /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-8m8bbyhv/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): ./vedio_distance/.DS_Store in function 'icvExtractPattern'


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


pth : .DS_Store


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


pth : 18.MOV


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will 

...

/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will 

pth : 9.MOV


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp9998
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_47516/2931198524.py:57: FutureWarning: The frame.append method is deprecated and will 

SST - 1 : mean : 1.152698412698413 ,result : 1.4689587301587381
SST - 3 : mean : 2.963085106382979 ,result : 0.12809468085106288
SST - 4 : mean : 4.094999999999999 ,result : 0.6858999999999846
SST - 7 : mean : nan ,result : 0
SSE - mean : 1.152698412698413 ,result : 0.004441269841269847
SSE - mean : 2.963085106382979 ,result : 0.5116053191489359
SSE - mean : 4.094999999999999 ,result : 1.2771000000000006
SSE - mean : nan ,result : 0
SSR- 1 : result : 1.4733999999999987
SSR- 3 : result : 0.6396999999999995
SSR- 4 : result : 1.963
SSR- 7 : result : 0
SST: 2.2829534110097853
SSE: 1.7931465889902063
SSR: 4.0760999999999985
rs 라이브러리 1m : 0.9871525986147935
++++++++++++
SSE/SST : 0.7854503645771159
1- (SSR/SST) : -0.7854503645771189
1-(SSE/SST) : 0.21454963542288408
---SST 값 변경----
SSE/SST : 5.0760999999999985
1- (SSR/SST) : -5.3492549249943995
1-(SSE/SST) : -4.0760999999999985


/Users/idahyeon/q/envs/newenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/idahyeon/q/envs/newenv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# 번호판으로 구하기

In [40]:
def getSST(m,results):
    result=0
    mean = np.mean(results)
    for r in results:
        result+= math.pow((m-mean),2)
    print("SST - "+str(m)+" : mean : "+ str(mean) + " ,result : "+str(result))
    return result

def getSSE(results):
    mean = np.mean(results)
    result=0
    for r in results:
        result+= math.pow((r-mean),2)
    print("SSE - mean : "+ str(mean) + " ,result : "+str(result))
    return result

def getSSR(m,results):
    result=0
    for r in results:
        result += math.pow((m-r),2)
    print("SSR- "+str(m)+" : result : "+str(result))
    return result


one_result=[]  #1m
three_result=[] #3m
five_result=[] #5m
seven_result=[] #7m

one = [str(i) for i in range(1,9)]
three = [str(i) for i in range(9,17)]
five = [str(i) for i in range(17,25)]
seven = [str(i) for i in range(25,33)]

pre_result = {}

vedios = os.listdir('./data/')
for pth in vedios:
    print("==================")
    print("pth : " + str(pth))
    
    cap = cv2.VideoCapture("./data/"+str(pth))
    
    reader = easyocr.Reader(['ko']) #이미지 -> 텍스트화 
    
    while cv2.waitKey(1) <0:
        hasFrame, frame = cap.read()
        pred_df = pd.DataFrame()
        
        if not hasFrame :   #프레임 끝났을 경우 종료 
            # print("Done Processing !!")
            cv2.waitKey(3000)
            break
        
        pred = yolov5(frame, size=1280, augment=False) # 번호판 인식  
        
        for i, row in pred.pandas().xyxy[0].iterrows(): #객체 정확도 분별
            if row['confidence'] < yolov5.conf: break
            pred_df = pred_df.append(row)
            
        pred_df = pred_df.reset_index()
        pred.save()
        
        
        for i, row in pred_df.iterrows(): # 한 프레임에서 인식한 객체 
            
            img = np.asarray(frame)
            cnt=0
            imageText=img.copy()
            img_cropped = img[int(row['ymin']):int(row['ymax']), int(row['xmin']):int(row['xmax'])]
            img_gray = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2GRAY)
            plt.subplot(4,4,i+1)
            result_text = reader.readtext(img_gray,detail =0)
        
            
            for res in result_text :
                res =res.replace(" ","")
                result_chars = ""
                type=0
                count=0
                for c in res:
                    if (ord('가') <= ord(c) <= ord('힣')):
                        if(count==2): 
                            result_chars +=c
                            type=2
                        elif(count==3):
                            if(type==2):
                                    break
                            result_chars +=c
                            type=3
                        else:
                            break
                    elif c.isdigit():
                        result_chars += c
                    else :
                        break
                    count+=1
                
                if(result_chars.isdigit() or result_chars in result): 
                    continue
                elif((type==2 and len(result_chars)==7) or (type==3 and len(result_chars)==8)) : 
                    if result_chars[-4:] in pre_result:
                        #거리구하기
                        area = (row['ymax']-row['ymin']-8)*(row['xmax']-row['xmin']-8)  #넓이
                        distance = round(135/math.sqrt(area),2)  #135 : 상헌상수
                        
                        if any(x in pth for x in seven) : 
                            seven_result.append(distance)
                        elif any(x in pth for x in five) : 
                            five_result.append(distance)
                        elif any(x in pth for x in three) : 
                            three_result.append(distance)
                        elif any(x in pth for x in one) : 
                            one_result.append(distance)
                        else : print("error pth")
                    elif i==0:
                        pre_result[result_chars[-4:]]=result_chars
            
            
            
    cap.release()

SST = getSST(1,one_result)+getSST(3,three_result)+getSST(5,five_result)+getSST(7,seven_result)
SSE = getSSE(one_result)+getSSE(three_result)+getSSE(five_result)+getSSE(seven_result)
SSR = getSSR(1,one_result)+getSSR(3,three_result)+getSSR(5,five_result)+getSSR(7,seven_result)

# SST = getSST(1,one_result)
# SSE = getSSE(one_result)
# SSR = getSSR(1,one_result)

print("SST: "+ str(SST))
print("SSE: "+ str(SSE))
print("SSR: "+ str(SSR))


R_Squared1= SSE/SST
R_Squared2 = 1 - (SSR/SST)

print("SSE/SST : "+str(R_Squared1))
print("1- (SSR/SST) : "+str(R_Squared2))

print("1-(SSE/SST) : "+str(1-(SSE/SST)))

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


pth : 21.MOV


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp4471
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp4472
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp4473
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will 

pth : 20.MOV


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp4548
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  p

...

/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp6563
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp6564
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp6565
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will 

pth : 10.MOV


/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp6632
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp6633
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_df = pred_df.append(row)
Saved 1 image to runs/detect/exp6634
/var/folders/24/n4yynsdj52ddzjt_t8m00zsr0000gn/T/ipykernel_17357/2243902946.py:59: FutureWarning: The frame.append method is deprecated and will 

SST - 1 : mean : 1.196180698151951 ,result : 18.743103901437397
SST - 3 : mean : 3.2833903133903135 ,result : 28.188834472934403
SST - 5 : mean : 3.997906976744186 ,result : 43.180188372093
SST - 7 : mean : nan ,result : 0
SSE - mean : 1.196180698151951 ,result : 2.264096098562622
SSE - mean : 3.2833903133903135 ,result : 18.416865527065536
SSE - mean : 3.997906976744186 ,result : 76.222511627907
SSE - mean : nan ,result : 0
SSR- 1 : result : 21.007199999999997
SSR- 3 : result : 46.605700000000056
SSR- 5 : result : 119.40270000000001
SSR- 7 : result : 0
SST: 90.1121267464648
SSE: 96.90347325353515
SSR: 187.01560000000006
SSE/SST : 1.075365511305467
1- (SSR/SST) : -1.0753655113054683
1-(SSE/SST) : -0.07536551130546698


/Users/idahyeon/q/envs/newenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/idahyeon/q/envs/newenv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
